### Understanding and cleaning Data:

**Tools:**

In [1]:
import time                                                 # para tiempo
inicio=time.time()                                          # momento inicial

   
import pandas as pd                                         # dataframe
import numpy as np                                          # numerical python, algebra lineal

import matplotlib.pyplot as plt                             # plots, graficos
import seaborn as sns                                       # plots

#from sklearn.metrics import confusion_matrix                # metricas, matriz de confusion
#from scipy.optimize import minimize                         # minimizar, opt


import warnings                                             # avisos
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [2]:
from sklearn.model_selection import train_test_split as tts
from sklearn.decomposition import PCA

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.metrics import mean_squared_error as mse

In [3]:
from sklearn.linear_model import *
from sklearn.linear_model import LinearRegression as LinReg

from sklearn.linear_model import LinearRegression as LinReg
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet

In [4]:
def highnull(x):
    return x.isnull().mean() * 100
    

### Limpieza de Nans

In [5]:
def delete_nan(df):
    data_1 = df.loc[:, df.isnull().mean() >= .5]
    #data.dropna(thresh=len(data)//2,axis=1, inplace=True)
    #trash_1 = list(data.loc[:, data.isna().sum() >= 1000])
    print(data_1)
    return df.drop(data_1,axis=1,inplace=True)

### Limpieza de columnas:

In [6]:
def trash_d(df):
    df_col = df.columns
    trash = []
    for i in df_col:
        if 'url' in i:

            trash.append(i)
        elif 'id' in i:

            trash.append(i)
        elif 'name' in i:

            trash.append(i)
        elif 'scrape' in i:

            trash.append(i)
        elif 'license' in i: #is it relevant??

            trash.append(i)
        elif 'latitude' in i:

            trash.append(i)
        elif 'longitude' in i:

            trash.append(i)

    print(len(trash))
    return df.drop(trash,axis=1,inplace=True)
       

### Analizar la correlacion entre las columnas y eliminar las que tienen corr > 90%

In [7]:
def bin_corr_col(x):
                                        
    corr_matrix = x.corr().abs()# Create correlation matrix
                                        
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)) # Select upper triangle of correlation matrix
    
    to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]# Find features with correlation greater than 0.90

    return x.drop(to_drop, axis=1, inplace=True) # Drop features

### Funcion para limpiar columna de 'Bathroom text'

In [8]:
import re

patron='[0-9]+'

def limpieza_ba(x):
    n_ba=0
    
    try:
        n_ba=int(re.findall(patron, x)[0])
    except:
        n_ba=0
        
    return n_ba

### Funcion para probar Modelo

In [9]:
def prueba_modelo(modelo):

    modelo.fit(X_train, y_train)

    y_pred=modelo.predict(X_test)

    train_score=modelo.score(X_train, y_train)  
    test_score=modelo.score(X_test, y_test)
    
    print(modelo)
    print('Train:', train_score)
    print('Test:', test_score) 
    print('\n')

**General Function**

In [10]:
def general_function(df):
    
    df_num = df.copy()
    
    df_num_1 = df_num._get_numeric_data()
    
    #Delete columns with 50% or more Nan values.
    #df_num_2 = df_num_1.apply(delete_nan)
    
    #Delete trash columns.
    df_num_2 = df_num_1.apply(trash_d)
    
    #Delete columns with higher correlation of 90%
    df_num_3 = df_num_2.apply(bin_corr_col)
    
    sns.heatmap(df_num_3.corr());
    
    return df_num_3   

### 1.Train CSV:

### 1er Modelo, con RFR, Filtrado, 
* get numeric, 
* simple trash,
* basic corr 90%
* Nos da mse de 90

**Import Train Data** 

In [11]:
df_train = pd.read_csv('data/train.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/train.csv'

In [ ]:
df_TN = df_train._get_numeric_data()

In [ ]:
trash_d(df_TN)


In [ ]:
delete_nan(df_TN)


In [ ]:
bin_corr_col(df_TN)

In [ ]:
# ELIMNAR COLUMNAS CON Nan, no tener miedo a eliminarlas!
df_TN.fillna(0, axis=0, inplace=True)


### Add Bathroom numeric column 

In [ ]:
df_TN['bathrooms']= df_train.bathrooms_text.apply(limpieza_ba)

In [ ]:
df_TN.columns

In [ ]:
### All this columns a have a variance ratio very far from one, i will delet them, and add, the bathroom column
columns_1 = ['beds','minimum_maximum_nights',
       'maximum_maximum_nights', 'availability_30', 'availability_365',
       'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d','review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month']

In [ ]:
df_TN.drop(columns_1, axis=1, inplace=True)

In [ ]:
corr_matrix = df_TN.corr().abs()# Create correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)) # Select upper triangle of correlation matrix
to_drop = [column for column in upper.columns if any(upper[column] > 0.80)]# Find features with correlation greater than 0.90
to_drop
    #return x.drop(to_drop, axis=1, inplace=True)

### Have a look to the correlation of the columns;

In [ ]:
sns.heatmap(df_TN.corr());

In [ ]:
# x.drop(to_drop, axis=1, inplace=True)

In [ ]:
df_TN.head()

In [ ]:
df_TN.shape

In [ ]:
#df_TN.drop(columns_1, axis=1, inplace=True)

### PCA 

In [ ]:
pca=PCA()

pca.fit(df_TN)

In [ ]:
pca.explained_variance_ratio_ # Mas cercano a cero
# 

In [ ]:
sum(pca.explained_variance_ratio_)

In [ ]:
df_TN.shape

In [ ]:
#sns.heatmap(df_TN.corr())

In [ ]:
df_TN.shape

In [ ]:
X=df_TN.drop('price', axis=1)
y=df_TN.price

In [ ]:
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.9, random_state=42)

### Definicion y activcion del modelo:

In [ ]:

import warnings                                             # avisos
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [ ]:
modelo=RFR()

modelo.fit(X_train, y_train)

### R2

In [ ]:
modelo.score(X_test, y_test)# R2 cuanto mas cercano a 1 mejor 

In [ ]:
y_pred=modelo.predict(X_test)

### MSE (ERROR CUADRATICO MEDIO) 1 No confiar & 0.85746 Muy bien & 0.24343445 Regular

In [ ]:
mse(y_pred, y_test, squared=False)

### 2. Test CSV:

**Import Test Data**

In [ ]:
new_data=pd.read_csv('data/test.csv')
new_data.shape

In [ ]:
df_test = new_data._get_numeric_data()

In [ ]:
delete_nan(df_test)
df_test.shape

In [ ]:
trash_d(df_test)


In [ ]:
bin_corr_col(df_test)

In [ ]:
df_test.shape

In [ ]:
df_test['bathrooms']= new_data.bathrooms_text.apply(limpieza_ba)

In [ ]:
df_test.isna().sum()

#### Hacer Fillna with mean... try

In [ ]:
df_test.drop(columns_1, axis=1, inplace=True)

In [ ]:
def fill_nan(df):
    
    for i in df.columns:
        
        df.fillna(df[i].mean(), axis=0, inplace=True)

    print(df_test.isna().sum())

In [ ]:
fill_nan(df_test)

In [ ]:
corr_matrix = df_test.corr().abs()# Create correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)) # Select upper triangle of correlation matrix
to_drop = [column for column in upper.columns if any(upper[column] > 0.80)]# Find features with correlation greater than 0.90
to_drop
    #return x.drop(to_drop, axis=1, inplace=True)

### Have a look to the correlation of the columns;

In [ ]:

sns.heatmap(df_test.corr());

In [ ]:
df_test.shape,df_TN.shape

### Eliminamos columns

In [ ]:
#df_test.drop(columns_1, axis=1, inplace=True)

### 3. Model Testing in Test Data

**Using RFR on Test Data**

In [ ]:
y_pred=modelo.predict(df_test)

In [ ]:
y_pred.shape

In [ ]:
f"Train : {modelo.score(X_train, y_train)} Test: {modelo.score(X_test, y_test)}"



In [ ]:
sample_df=pd.DataFrame()


sample_df['id']=[str(i) for i in range(len(y_pred))]
sample_df['price']=y_pred

sample_df.shape

In [ ]:
sample_df.head()

In [ ]:
#sample_df.to_csv('data/sample_df.csv',index=False)

In [ ]:
def prueba_modelo(modelo):

    modelo.fit(X_train, y_train)

    y_pred=modelo.predict(X_test)

    train_score=modelo.score(X_train, y_train)  
    test_score=modelo.score(X_test, y_test)
    
    print(modelo)
    print('Train:', train_score)
    print('Test:', test_score) 
    print('\n')


In [ ]:
modelos=[LinReg(), Lasso(), ElasticNet(), SVR(),RFR()]

for m in modelos:
    prueba_modelo(m)

In [ ]:
modelos=[LinReg(), Lasso(), ElasticNet(), SVR(),RFR()]
dict_fit = {}
for k,v in enumerate(modelos):
    dict_fit[k]=prueba_modelo(v)

In [ ]:
dict_fit